In [ ]:
getwd()


In [ ]:
players_url <- "https://raw.githubusercontent.com/RaeL88/DSCI_project/refs/heads/main/players.csv"

In [ ]:
sessions_url <- "https://raw.githubusercontent.com/RaeL88/DSCI_project/refs/heads/main/sessions.csv"

In [ ]:
library(tidyverse)

In [ ]:
players <- read_csv(players_url)
head(players)


In [ ]:
sessions <- read_csv(sessions_url)
head(sessions)

In [ ]:
set seed
HTML
run all cells from top to bottom before submitting
after you download, check for visualizations and all steps visible 

In [ ]:
Data Description

Number of Observations:

Players data set = 196 observations
Sessions data set = 1535 observations

Summary statistics (piazza said we can write observations
                    in the description of the data 
                    sets, then find max etc. in part 3):
Players data
- Participants are aged 9 through 50
- Genders include male, female, non-binary, agender, 
  and two-spirited
- Some participants chose not to state their gender
- Hours played ranged from 0 to 223.1
- Experience consisted of factors, amateur, beginner,
  regular, pro, and veteran

Sessions data
- The year of the start time is 2024 for all participants
- It contains observations about the play sessions for each
  participant
- It contains the start and end times for each player



Number of variables:

Players data = 7 variables

Sessions data = 5 variables



Name and type of variables:

Players data

Variable    |  Type
____________|______________
experience  | character
____________|______________
subscribe   | logical
____________|______________
hashedEmail | character
____________|______________
played_hours| double
____________|______________
name        | character
____________|______________
gender      | character
____________|______________
Age         | double
____________|______________

Sessions data (contains the same hashedEmail variable as above)

Variable    |  Type
____________|______________
start_time  | character
____________|______________
end_time    | character
____________|______________
hashedEmail | character
____________|_______________________
orginal_start_time   | double
_____________________|______________
orginal_end_time     | double
_____________________|______________

What variables mean:
experience: The level of skill the participant has in Minecraft

subscribe: Whether the participant has subscibed to the game-related newsletter

hashedEmail: A string of characters representing the email of a participant 

played_hours: The total number of hours a participant has played Minecraft

name: The name of a participant 

gender: The gender of a participant 

Age: The age of the participant

start_time: The time the participant has started playing Minecraft

end_time: The time the participant has stopped playing Minecraft

original_start_time: The milliseconds that have passed since January 1st, 1970 (Unix epoch time)

original_end_time: The milliseconds that have passed since January 1st, 1970 (Unix epoch time)


Issues in data:

Any other potential issues related to things 
you cannot directly see (how data was collected etc.)

- The data might be self-reported, which can make the data prone to biases/inaccuracies.
- They might have sampled more of certain age groups or genders which could make their
  data or findings disproportionate/also biased. 
- They might have taken more data from of a certain age or age group than from others,
  which can interfere with the predictive question I am proposing. 
- The model might not be able to predict hours played of a participants with a less common age in the data
  set as well. 
  


In [ ]:
Broad Question: Question 2 (We would like to know which "kinds"
                            of players are most likely to contribute
                            a large amount of data so that we can target
                            those players in our recruiting efforts)

Specific Question: What gender and age group plays the longest hours? 

How the data will address the question of interest 
(The plan for wrangling to apply a predictive method 
 from class):

The data will find the gender and age groups that play the most hours, 
so that they can be recruited as they contribute a large amount of data. 

The plan is to use multivariable regression in order to predict
the hours a participant of a certain gender and age
group will play. 

The players data set will be used. 



In [ ]:
library(tidyverse)

In [ ]:
#1) Make data into tidy format

#change experience in players data into factors


#make start date and end date columns to split them from start and end time for sessions

#to tidy players data

tidy_players <- players |> mutate(experience = as_factor(experience)) 



head(tidy_players)



#To tidy sessions data:

tidy_sessions <- sessions |> separate(col = start_time, into = c("start_date", "start_time"), sep = "\ ") |> 
separate(col = start_time, into = c("start_hour", "start_minute"), sep = ":") |>
separate(col = end_time, into = c("end_date", "end_time"), sep = "\ ") |>
separate(col = end_time, into = c("end_hour", "end_minute"), sep = ":") |> select(hashedEmail:end_minute) 
head(tidy_sessions)

#2) Compute the mean, median, max, min, sd value for each quantitative varible



summary_players <- tidy_players |> select(played_hours, Age) |> summary()
summary_players


sd_players_hours <- players |> select(played_hours) |> pull() |> sd(na.rm = TRUE)

sd_players_Age <- players |> select(Age) |> pull() |> sd(na.rm = TRUE)

sd_players_hours
sd_players_Age



#3) Make a few exploratory visualizations of the data

#scatterplot of age and played_hours
options(repr.plot.width = 12, repr.plot.height = 15) 
players_plot1 <- players |> ggplot(aes(x = Age, y = played_hours, fill = gender)) + geom_bar(stat = "identity") + labs(x = "Age of participants (Year)", 
y = "Hours Played", fill = "Gender", title = "Relationship between age and hours of Minecraft played") +
theme(text = element_text(size = 20))
players_plot1

#how many of each age are playing Minecraft

age_count <- players |>
    group_by(Age) |>
    summarise(count = n()) 
age_count

age_count_chart <- age_count |> ggplot(aes(x = Age, y = count)) + geom_bar(stat = "identity") +
labs(x = "Age of Participants (Years)", y = "Number of Participants from a Certain Age", 
      title = "Number of Participants per Age") + theme(text = element_text(size = 20))

age_count_chart



#Insights from Plot
#- More participants around age 8 to about ~28 play than participants past 30.
#- Participants around age 8 to ~28 are likely to play for a prolonged period of time compared to participants
#  older than 30 
#- Most of the data came from participants older than 10 and less than 30 
#- The most abundant gender in the data set is male





In [ ]:
Method to address question of interest: 
multivariable knn regression will be used because the data does not look linear

1) What data set was chosen

- The players data set was chosen

2) Why the data set was chosen

-It contains the variables, played_hours, gender, and age which are needed to answer my specific question. 

3) Why this method is appropriate

-This method is appropriate this data set is not linear, and multivariable linear regression is for linear 
- This method can predict numerical labels, so we can predict hours played

4) Which assumptions are required, if any, to apply
   the method selected?

- that if points are close to each other (neighbours) they are similar
  and can predict the label of a new data point. 


5) What are the potential limitations or weaknesses
   of the method selected?

- if the scale of each vairable is far apart from another, 
  it can affect the variables with the smaller scale 
-if the data is underfitted (not enough neighbours)
or overfitted (too many neighbours), the predicted
values will be affected

6) How are you going to compare and select the model?

- Using cross validation to choose K 
-RMSPE equation to predict error from each observation
to the predicted data 
-we choose the K corresponding to the lowest RMSPE 
-training sets and testing sets

7) How are you going to process the data to apply it
   to the model? (ex. how many splits, what proportions 
                  will you use for the splits, at which
                  stage will you split, will there be a 
                  validation set? Will you use cross
                  validation? 

- 5 splits will be used
- 75% of the data will be used for training, 25% for testing
- The data will be split after it is standardized and the spec 
  is made
- After the split, the workflow will be made
          